# Helium turbine aerodynamic design program

This document is intended as an in-depth guide to the helium turbine performance prediction functions written by WD. Although the code is identical to that in accompanying files, it should not be run, instead it should be used as a refernece in conjunction with the main files turbine.py, prof_gen.py and others. Comments from these files have been left in to further aid understanding of the admittedly rather messy code. I would suggest that should you wish to reconstruct the functionality of this program that you take some time to plan and possibly take a more object oriented approach rather than recreating functions with 15+ arguments and outputs and 100+ local variables!

The aim of this code is to model the aerodynamic performance of the helium turbine used in Reaction Engines' SABRE, which operates in an unconventional design space. While efforts have been made to generalise prediction ability, it should be noted that the methods applied here may not be compatible with significantly different specifictions, particularly the methods associated with gas properties, loss mechanisms and thermo-mechanical behaviour. 

In general, the program uses basic turbomachinery principles to take design parameters and convert them to a complete turbine geometry, from which performance can be predicted with the use of correlations for loss mechanisms. It is possible to vary stage parameeters through the machine using splines and a number of control points, which can then be fed into optimisation routines to seek optimal performance. Machine is predicted by considering stress constraints, and resulting geometries can then be fed into further mechanical modelling, see ZK's work. 

The program makes significant use of the numpy, scipy and math libraries, and also uses cvs and matplotlib for some optional functionalities. The provided folder should contain the following files:

- turbine.py - contains all of the functions for turbine geometry and performance prediction aside from those associated with blade profiles. Also contains a function for optimisation of a turbine design.
- prof_gen.py - contains functions for the generation of blade profiles written by CJC. Slight modifications have been made by WD, as well as the addition of a function for calculating blade section area and suction surface length and a function for tabulating these in .csv files.
- GUI.py - functions for plotting the blades and meridional views of turbines. Also contains a function that generates a GUI with matplotlib that can be used for rapid visualisation and testing of turbine designs. This GUI is quite slow and should be ported to a library more suited to GUI design. 
- Profile_areas.csv and ss_grid.csv - tabulated data from the prof_gen functions, used when speed is necessary as interpolatiomn from these is faster than generating new sections every time. 
- Data Mining.py, Plots.py - both conatin none of the prediction code, but have been used to investigate behaviours. Data Mining is where the functions from turbine.py were generally run and contains code to set inputs and print outputs and for extracting data for things like mesh generation with Autogrid. Also contains script for various optimisation routines and a sensitivity analysis. Plots as it suggests is where scripts for plotting relevant results have been kept. Both of these files are extremely messy and can be replaced with no loss in functionality. 

What follows will be a detailed explaination of each function. If you have any more questions about what something does email wd269@cam.ac.uk. If any of the code looks to be formatted in a strange way it has likely been done so to conform with the python style guide set out in [PEP 8](https://www.python.org/dev/peps/pep-0008/#blank-lines).

# turbine.py

### Preamble

Basic things like docstring and imports. Also set here are some global variables:
- print_warnings sets whether or not you want to print the warnings that the stress calculations have found the given design would fail. This should generally be set to False as it can throw up a lot of warnings. 
- fast_dimensions sets the use of the tabulated data for blade section areas and suction surface lengths. The difference  between interpolation and direct calculation is quite small, so this should be True unless you want to do precise individual calculations as the interpolation is much faster, which helps with optimisation. 
- transient_analysis sets if you want to do a full transient analysis for the warm up expansion of a stage. This should be False unless you are specifically investigating that.
- plotting is a list of the stages for which you wish to plot the transient expansion analysis. 

In [ ]:
"""
Module contains functions to evaluate helium turbine performance.
Helium gas properties set in turbine function, would require new values of
cp and gamma and a new viscosity law
"""
# Import required modules
import csv
import numpy as np
import scipy.integrate as integrate
import matplotlib.pyplot as plt
from scipy import interpolate as sciint
from scipy.optimize import minimize
from prof_gen import blade_dims

# Declare a few global options
# If True, will print warnings when material limits are exceeded
print_warnings = False
# If True, will interpolate from tables for blade dimensions
fast_dimensions = True
# If True, will perform full transient analysis on start-up
transient_analysis = False
# Stage number to plot transient, otherwise no plotting
plotting = [9]

### Turbine

The turbine function encapsulates all the overall turbine design and should be the only function that you need to interact with in design, aside from the optimisation function if needed. As such, the inputs are the only things you should need to fully specify the design. These are:

- Inlet stagnation pressure $P_{01}$ in Pa and stagnation temperature $T_{01}$ in K
- Mass flow rate $\dot{m}$ in kg/s
- Rotational speed $\Omega$ in rad/s
- Power output $W$ in W
- Trailing edge thickness $t$ and shroud clearance $g$, both in m
- Flow coefficient $\phi$, stage loading coefficient $\psi$ and reaction $\Lambda$
- Blade aspect ratio $AR$
- Stage work distribution $dh_0$, which is a list that scales the work by each stage i.e. [1, 2] would mean the last stage does double the work of the first.
- Number of stages $n$
- Pitch-to-chord ratio $p/C_x$
- Inlet flow angle $\alpha_{in}$
- Working gas, currently only helium (He) and the properties of burning A1 jet fuel at an AFR of 50 (A1)

In [3]:
########################
# MAIN TURBINE ROUTINE #
########################


def turbine(Po1, To1, mdot, Omega, W, t, g, phi, psi, Lambda, AR, dho,
            n, ptc=-1, ain=0, gas='He'):
    """Return the turbine performance and sizing"""

The next part conditions the inputs to ensure they make sense and will work with the spline function. This means that they need to be iterable, meaning any int or float must be converted to a list, and if the input iterable is longer than the number of stages it is cut down to size.

In [ ]:
    # If the inputs for phi, psi, Lambda, dho, AR, or ptc are ints
    # or floats, assume they'reconstant through the turbine
    if isinstance(phi, (float, int)):
        phi = [phi, phi]
    if isinstance(psi, (float, int)):
        psi = [psi, psi]
    if isinstance(Lambda, (float, int)):
        Lambda = [Lambda, Lambda]
    if isinstance(dho, (float, int)):
        dho = [dho, dho]
    if isinstance(AR, (float, int)):
        AR = [AR, AR]
    if isinstance(ptc, (float, int)):
        ptc = [ptc, ptc]
    # Ensure the inputs aren't longer than the number of stages,
    # if they are cut them off
    if len(phi) > n:
        phi = phi[:n]
    if len(psi) > n:
        psi = psi[:n]
    if len(Lambda) > n:
        Lambda = Lambda[:n]
    if len(dho) > n:
        dho = dho[:n]
    if len(AR) > n:
        AR = AR[:n]
    if len(ptc) > n:
        ptc = ptc[:n]

The program can handle both repeating and non-repeating stages, but these have different functions for finding flow angles. To determine which to use check if all the input parameters are the same through the machine, indicating repeating stages, and set the function required. We also store the inputs here so that they can be output, this is a slightly janky thing done for the GUI. 

In [ ]:
    # If all loadings are constant treat as repeating stages
    rep = False
    if (all(i == phi[0] for i in phi) and all(i == psi[0] for i in psi)
            and all(i == Lambda[0] for i in Lambda)):
        find_angs = repeating
        rep = True
        ain = np.degrees(find_angs(phi[0], psi[0], Lambda[0], ain)[0])
    else:
        find_angs = angles
    # Store the inputs for later use
    inits = [Po1, To1, mdot, Omega, W, t, g, phi, psi,
             Lambda, AR, dho, n, ptc, ain, gas]

Now the parameters are determined for every stage by interpolating with the control points. $dh_0$$ is scaled to ensure the correct amount of total work is done, and the parameters are stored to be unput to the stages. 

In [ ]:
    # Create splines of the stage loadings if n>1
    if n > 1:
        phi = spline(n, phi)
        psi = spline(n, psi)
        Lambda = spline(n, Lambda)
        dho = spline(n, dho)
        AR = spline(n, AR)
        ptc = spline(n, ptc)
        # Scale the enthalpy drops to ensure the total work is as desired
        sum_dho = sum(dho)
        dho = [W/(mdot*sum_dho)*i for i in dho]
    # If n=1 then scale the first vale of dho accordingly
    if n == 1:
        dho = [W/mdot, 0]
    # List of the stage parameters
    stage_params = [phi, psi, Lambda, dho, AR, ptc]

A limit on blade exit angles has been set at 73º to ensure machinability and correlation validity. Here the angles are checked and an output is set to warn if an angle surpasses the limit. 

In [ ]:
    # Set hard limit of 73º on exit angles. Don't account for increase in psi
    # due to leakage here, can remove designs if they're unacceptable later.
    ang_1 = 0  # Initialise the inlet angle
    angle_warning = [False, 0]  # Initialise the angle warning
    for ph, ps, La in zip(phi, psi, Lambda):
        ang_check = find_angs(ph, ps, La, ang_1)
        ang_2 = np.degrees(ang_check[1])
        ang_3 = np.degrees(ang_check[4])
        ang_1 = ang_check[3]
        if np.round(abs(ang_2), 2) > 73.0 or np.round(abs(ang_3), 2) > 73.0:
            angle_warning = [True, max(abs(ang_2), abs(ang_3))]
        elif not angle_warning[0]:
            angle_warning = [False, max(abs(ang_2), abs(ang_3))]

Desired metrics are initialised here to be stored and output. Also set here is the material used for the hub (rotor) and case (stator). 

In [ ]:
    # Rotor and stator materials
    rotor_mat = 'inconel'
    stator_mat = 'inconel'
    materials = [stator_mat, rotor_mat]
    # Overall specific enthalpy drop
    del_ho = W/mdot
    # Initialise quantities
    Poi = Po1
    Toi = To1
    a1i = np.radians(ain)
    loss = 0
    loss_comp = [0, 0, 0, 0]  # Entropy from profile, TE, secondary and tip
    length = 0
    volume = 0
    mass = 0
    work = 0  # To check that total work adds up to required work
    dims = []  # Array containing turbine dimensions
    angle = []  # Stage angles
    n_blades = 0  # Total blades in the turbine
    Fx = 0  # Axial force on rotor
    Re = 0  # Average Reynolds number
    expansion_lims = []  # Stator and rotor expansion and thermal time

If using the interpolation for blade profile dimensions i.e. fast_dimensions == True, the csv files need to be read to create functions for finding them given blade angles. The csv library is used to read the files and sciint.RectBivariateSpline creates interpolation functions which are stored for passing to the next function. 

In [ ]:
    # Select the function for calculating blade
    # dimensions depending on speed wanted
    global fast_dimensions
    dim_fs = []
    if fast_dimensions:
        # Load the grid of normalised suction surface lengths
        # and create the interpolation function
        xin = []
        xout = []
        sslen = []
        areas = []
        with open("ss_grid.csv") as csvfile:
            reader = csv.reader(csvfile, quoting=csv.QUOTE_NONNUMERIC)
            line = 0
            for row in reader:
                if line == 0:
                    xout = row[1:]
                    line += 1
                else:
                    xin.append(row[0])
                    sslen.append(row[1:])
                    line += 1
        with open("Profile_areas.csv") as csvfile:
            reader = csv.reader(csvfile, quoting=csv.QUOTE_NONNUMERIC)
            line = 0
            for row in reader:
                if line == 0:
                    line += 1
                else:
                    areas.append(row[1:])
                    line += 1
        # Make the lists numpy arrays for the interpolation
        xout = np.asarray(xout)
        xin = np.asarray(xin)
        sslen = np.asarray(sslen)
        areas = np.asarray(areas)
        # Make the interpolation functions
        ss_length = sciint.RectBivariateSpline(xin, xout, sslen, kx=1, ky=1)
        profile_area = sciint.RectBivariateSpline(xin, xout, areas, kx=1, ky=1)
        dim_fs = [ss_length, profile_area]

We now start looping over each stage. For each stage we use the parameters to find the radius, blade speed and axial velocity. Gas properties are calculated and all stage parameters are packaged up to pass on to the stage function.

In [ ]:
    # Increment through every stage
    i = 0
    while i < n:
        # Mean radius and blade speed, constant throughout
        r = np.sqrt(dho[i]/(psi[i]*Omega**2))
        U = r*Omega
        Vx = U*phi[i]  # Axial velocity fixed by turbine parameters
        # Create size array, including the function for ss length
        sizes = [t, g, r, dim_fs]
        # Gas properties
        gamma, cp, R = thermo_props(Toi, gas)
        gas_props = [cp, gamma, R, gas]
        # Create array of parameters needed for the stage
        params = [mdot, Omega, U, Vx, psi[i], phi[i], Lambda[i], a1i,
                  AR[i], ptc[i], rep, i+1, n]

The performance of the stage is calculated and stored. The inlet conditions for the next stage are also stored. Total loss is found, as well as the components from each mechanism. The stored Reynolds number is the average of that for the rotor and stator. 

In [ ]:
        # Update the input conditions for the next stage
        Poin = Poi
        Toin = Toi
        stage_calc = stage(Poin, Toin, dho[i], params, sizes,
                           gas_props, materials)
        # Store angles before updating a1
        angle.append([np.degrees(x) for x in
                      find_angs(phi[i], psi[i], Lambda[i], a1i)])
        # Find the exit conditions from this stage to pass to the next
        Toi = stage_calc[0]
        Poi = stage_calc[1]
        a1i = stage_calc[10]
        # Find the required quantities and store them
        loss += stage_calc[8]
        loss_comp[0] += stage_calc[9][0]  # Profile
        loss_comp[1] += stage_calc[9][1]  # Trailing edge
        loss_comp[2] += stage_calc[9][2]  # Secondary flow
        loss_comp[3] += stage_calc[9][3]  # Tip leakage
        length += stage_calc[6]
        volume += stage_calc[4]
        mass += stage_calc[3]
        work += stage_calc[5]
        n_blades += stage_calc[7][10]+stage_calc[7][11]
        Fx += stage_calc[11]
        dims.append(stage_calc[7])
        Re += (stage_calc[13][0]+stage_calc[12][1])/(2*n)
        expansion_lims.append(stage_calc[13])
        # Move to the next stage
        i += 1

Finally, the isentropic efficiency of the turbine is calculated and all required metrics are output. 

In [ ]:
    # Find the efficiency using the overall loss
    eff = del_ho/(del_ho+To1*loss)
    # More outputs can be added for whatever is needed
    return [eff, work*mdot, mass, volume, length, dims, n_blades, loss_comp,
            Poi, Toi, angle, inits, angle_warning, Fx, Re,
            expansion_lims, stage_params]

### Stage

The stage function handles calculations for each individual stage, which can then be called by the main turbine function. As arguments it takes the stage inlet conditions and all required parameters, defined in the turbine function. These are unpacked and reconfigured for use here. 1 refers to stage inlet, 2 is stator exit/rotor inlet and 3 is stage exit.

In [ ]:
#################
# STAGE ROUTINE #
#################


def stage(Po1, To1, del_ho, params, sizes, gas_props, materials):
    """Return the stage losses and exit conditions"""

    global fast_dimensions, print_warnings, transient_analysis, plotting
    # Gas properties
    cp, gamma, R, gas = gas_props
    # Stage parameters
    mdot, Omega, U, Vx, psi, phi, Lambda, a1, AR, ptc, rep, \
        stage_n, n_stages = params
    # Set the angle function:
    if rep:
        find_angs = repeating
    else:
        find_angs = angles
    # Major sizes
    t, g, r, dim_fs = sizes
    # Calculate angles
    a1, a2, b2, a3, b3 = find_angs(phi, psi, Lambda, a1)
    # Create array of angles needed in the loss function
    angs = [a1, a2, b2, b3]
    # Calculate velocities
    V1, V2, W2, V3, W3 = velocities([a1, a2, b2, a3, b3], Vx)
    # Create array of velocities needed in the loss function
    vels = [V1, V2, W2, W3]

Here static and stagnation temperatures through the stage are found, which can then be used to find the speed of sound and gas viscosity. Inlet static pressure is then found with the compressible gas relation. Mach numbers are generally low enough in the helium turbine that we could just use Bernoulli, however the compressible relation means we don't need density, which is calculated using static pressure. 

In [ ]:
    # Temperatures, speed of sound and viscosity through the stage
    T1 = To1 - 0.5*(V1**2)/cp
    c1 = np.sqrt(gamma*R*T1)
    To2 = To1  # No stagnation temperature drop over stator
    T2 = To2 - 0.5*(V2**2)/cp
    c2 = np.sqrt(gamma*R*T2)
    mu2 = viscosity(T2, gas)
    To3 = To2 - del_ho/cp  # Stagnation temperature drop depending on work
    T3 = To3 - 0.5*(V3**2)/cp
    c3 = np.sqrt(gamma*R*T3)
    mu3 = viscosity(T3, gas)
    # Use compressible relation to find static pressure
    P1 = Po1*(1+(gamma-1)*((V1/c1)**2)/2)**(-gamma/(gamma-1))
    rho1 = P1/(R*T1)

In order to find the pressure and density through the rest of the stage, we need to know the stagnation pressure loss. The problem is that the density will affect the stage dimensions, which in turn will affect the amount of stagnation pressure loss. The caculations are therefore iterated until convergence, generally only taking 2-3 loops. An initial guess of a 1% loss is used to calculate pressures and densities in the same way as before, and all the gas states needed in the loss correlations are stored in a list.

In [ ]:
    # Initialise loss coefficients
    Y_st_guess = 0.01
    Y_ro_guess = 0.01
    Y_st = 0
    Y_ro = 0
    # Iterate until the loss coefficients are stable
    while (abs(Y_st_guess-Y_st)/Y_st_guess > 0.001
           and abs(Y_ro_guess-Y_ro)/Y_ro_guess > 0.001):
        # Update the loss coefficients
        Y_st = Y_st_guess
        Y_ro = Y_ro_guess
        # Stator to rotor pressure change
        Po2 = Y_po(Po1, Y_st, V2, c2, gamma)  # Stagnation pressure after loss
        P2 = Po2*(1+(gamma-1)*((V2/c2)**2)/2)**(-gamma/(gamma-1))
        # Move into relative frame for the rotor
        Po2rel = P2*(1+(gamma-1)*((W2/c2)**2)/2)**(gamma/(gamma-1))
        rho2 = P2/(R*T2)
        # Exit
        Po3rel = Y_po(Po2rel, Y_ro, W3, c3, gamma)
        # Move out of the relative frame to find exit states
        P3 = Po3rel*(1+(gamma-1)*((W3/c3)**2)/2)**(-gamma/(gamma-1))
        Po3 = P3*(1+(gamma-1)*((V3/c3)**2)/2)**(gamma/(gamma-1))
        rho3 = P3/(R*T3)
        # Create array for the loss function
        states = [T1, T2, T3, rho1, rho2, rho3, mu2, mu3, mdot]

Now that density is known the stage can be sized. Blade heights are found using the mass flow and axial velocity. Blade chords are calculated with the average span and aspect ratio, and the throat width ($w$) is found as a function of chord, pitch-to-chord and angles. Also calculated are the suction surface lengths, which are the dimension used in the Reynolds number. If the interpolation function is being used then only the lengths are found here, but if they are being calculated directly from the profile generator then we want to minimise the number of calls, so both length and area are found here. The area is used in the stress calculations.

In [ ]:
        # Stage sizing
        H1 = mdot/(rho1*2*np.pi*r*Vx)
        H2 = mdot/(rho2*2*np.pi*r*Vx)
        H3 = mdot/(rho3*2*np.pi*r*Vx)
        # Use average blade span to find axial chord and throat width
        H_st = (H1+H2)/2
        H_ro = (H2+H3)/2
        Cx_st = H_st/AR
        Cx_ro = H_ro/AR
        w_st = p_w(Cx_st, a1, a2, ptc)[0]
        w_ro = p_w(Cx_ro, b2, b3, ptc)[0]
        # Create array for the loss function
        dimensions = [t, g, r, H_st, H_ro, Cx_st, Cx_ro, w_st, w_ro]
        # Calculate the Reynolds numbers
        # Run the profile generator for SS length and section area
        if fast_dimensions:
            ss_st = Cx_st*dim_fs[0](np.degrees(a1), np.degrees(a2))[0][0]
            ss_ro = Cx_ro*dim_fs[0](np.degrees(b2), np.degrees(b3))[0][0]
        else:
            stator_dims = blade_dims(np.degrees(a1), np.degrees(a2), t, Cx_st)
            rotor_dims = blade_dims(np.degrees(b2), np.degrees(b3), t, Cx_ro)
            stator_A = stator_dims[0]
            ss_st = stator_dims[1]
            rotor_A = rotor_dims[0]
            ss_ro = rotor_dims[1]
        # Calculate Re based on SS length
        Res = [rho2*V2*ss_st/mu2, rho3*W3*ss_ro/mu3]

Finally in this loop, loss is calculated using another function. With the loss new stagnation pressure loss coefficients can be found. The loss found here is entropy, however the entropy loss coefficient is equivalent to the stagnation pressure coefficient at low Mach numbers. 

In [ ]:
        # Find the losses across the stator and rotor
        loss_calc = losses(angs, vels, states, dimensions, Res)
        loss_st = loss_calc[1]  # Stator entropy rise
        loss_ro = loss_calc[2]  # Rotor entropy rise
        # Use entropy rise to find stagnation pressure loss coefficients
        # assuming entropy loss coefficient is equivalent at low Mach numbers
        Y_st_guess = loss_st*T2/(0.5*V2**2)
        Y_ro_guess = loss_ro*T3/(0.5*W3**2)

With stage dimensions and losses now fully specified a few other metrics are calculated. An 'actual' value of the work from the stage is found by assuming that all leakage flow does no work. In a previous version this was used in another loop to adjust stage loading to match the required work output, but it is now just something to take note of. Stage efficiency is found, as are the length and volume, assuming a quarter chord gao either side of each blade row. All dimensions are then stored. 

In [ ]:
    # Find the actual work output given the rotor leakage
    m_l = np.amax(loss_calc[4])
    work = psi*(1-m_l)*U**2
    # Stage performance
    loss_array = loss_calc[3]
    loss = loss_calc[0]
    eff = del_ho/(del_ho+To3*loss)
    # Stage sizes
    length = 1.5*Cx_st + 1.5*Cx_ro
    vol_st = (np.pi*Cx_st *
              (0.25*((r+H1/2)**2)+((r+(H1+H2)/4)**2)+0.25*(r+H2/2)**2))
    vol_ro = (np.pi*Cx_ro *
              (0.25*((r+H2/2)**2)+((r+(H2+H3)/4)**2)+0.25*(r+H3/2)**2))
    volume = vol_st + vol_ro
    ptc_st = p_w(Cx_st, a1, a2, ptc)[1]
    ptc_ro = p_w(Cx_ro, b2, b3, ptc)[1]
    dimensions.append(ptc_st)
    dimensions.append(ptc_ro)

The mass of the the stage is found with stress calculations, which also find the number of blades and the inner and outer radii of the drum and casing. Axial force is also found. 

In [ ]:
    # Stage mass and blades
    if fast_dimensions:
        stator_A = Cx_st**2*dim_fs[1](np.degrees(a1), np.degrees(a2))[0][0]
        rotor_A = Cx_ro**2*dim_fs[1](np.degrees(b2), np.degrees(b3))[0][0]
    blade_areas = [stator_A, rotor_A]
    mass_calc = stage_mass(To1, Po1, Po3, dimensions, Omega, blade_areas,
                           materials, stage_n, n_stages)
    mass = mass_calc[0]
    stator_blades_N = mass_calc[1]
    rotor_blades_N = mass_calc[2]
    Ro_st = mass_calc[3]
    Ri_ro = mass_calc[4]
    dimensions.append(Ro_st)
    dimensions.append(Ri_ro)
    # Axial force on rotor
    Fx = blade_force(P2, P3, r, H2, H3)

The change in tip clearance during start up, when the hub and case may expand at different rates as they start rotating and heat up, could affect the viability of a design. A basic lumped heat capacity calculation is done to estimate these changes. The time constant for heating (tau) is found, and the mass function gives the tip clearance when cold (dg_c) and hot (dg_h).

In [ ]:
    # Thermal start up
    vels.append(V3)
    pressures = [P1, P2, P3]
    thermal_calc = thermal(vels, states, pressures, dimensions, materials, gas)
    tau_st = thermal_calc[0]
    tau_ro = thermal_calc[1]
    dg_c = mass_calc[5][0]-mass_calc[5][1]
    dg_h = mass_calc[6][0]-mass_calc[6][1]

Without doing a full transient analysis, the worst case scenario tip gap during warm up can be estimated by assuming a linear relationship i.e. when one part is fully soaked, the other has reached a temperature depending on the ratio of time constants.

In [ ]:
    # The heating time constants can be used to get an estimate of a worst
    # case scenrio where one component is fully heated and the other only
    # heated to some fraction governed by the relative heating times
    if thermal_calc[0] < thermal_calc[1]:
        # Scenario where stator heats up faster
        dr_st = mass_calc[6][0]
        dr_ro = mass_calc[5][1]+tau_st/tau_ro*(mass_calc[6][1]-mass_calc[5][1])
        dg_warm = dr_st-dr_ro
    if thermal_calc[0] >= thermal_calc[1]:
        # Scenario where rotor heats up faster
        dr_ro = mass_calc[6][1]
        dr_st = mass_calc[5][0]+tau_ro/tau_st*(mass_calc[6][0]-mass_calc[5][0])
        dg_warm = dr_st-dr_ro

This part looks pretty grim, however it is necessary for each scenario. The tip clearances when cold-static (not rotating), cold-rotating, warm and hot will depend on what the initial machined clearance is. This has to be set such that at no point does the rotor rub against the case. For example the first case has the case always expanding more than the hub, so we set the machined (cold-static) clearance to be zero. 

In [ ]:
    # Depending on the relative values of the hot and cold strains, the tip
    # clearance when cold-static, cold-rotating, warm-rotating
    # and hot-rotating will change
    if dg_c >= 0 and dg_h >= 0:
        cold_stat_g = 0.0
        cold_rot_g = dg_c
        warm_rot_g = dg_warm
        hot_rot_g = dg_h
    elif dg_c >= 0 and dg_h < 0:
        cold_stat_g = abs(dg_h)
        cold_rot_g = dg_c+abs(dg_h)
        warm_rot_g = dg_warm+abs(dg_h)
        hot_rot_g = 0.0
    elif dg_h >= 0 and dg_c < 0:
        cold_stat_g = abs(dg_c)
        cold_rot_g = 0.0
        warm_rot_g = dg_warm+abs(dg_c)
        hot_rot_g = dg_h+abs(dg_c)
    elif dg_c < 0 and dg_h < dg_c:
        cold_stat_g = abs(dg_h)
        cold_rot_g = dg_c-dg_h
        warm_rot_g = dg_warm+abs(dg_h)
        hot_rot_g = 0.0
    elif dg_h < 0 and dg_c <= dg_h:
        cold_stat_g = abs(dg_c)
        cold_rot_g = 0.0
        warm_rot_g = dg_warm+abs(dg_c)
        hot_rot_g = dg_h-dg_c

If the setting for transient analysis has been set to True, the more detailed transient function is called to find the maximum tip clearance during warm up. This analysis calculates the expansion of the hub and casing in time rather than just scaling with the time constant. 

In [ ]:
    # Transient analysis if requested at the top
    if transient_analysis:
        trans = [True, tau_st, tau_ro]
        # This is not an ideal solution but as long as stage_mass is called
        # like this then it will just return the transient clearances
        dim_trans = (t, g, r, H_st, H_ro, Cx_st, Cx_ro,
                     w_st, w_ro, ptc_st, ptc_ro)
        trans_ans = stage_mass(To1, Po1, Po3, dim_trans, Omega, blade_areas,
                               materials, stage_n, n_stages, trans)
        g_trans = trans_ans[0]
        trans_start = cold_rot_g-g_trans[0]
        g_trans = [i+trans_start for i in g_trans]
        t_trans = trans_ans[1]
        warm_rot_g = max(g_trans, key=abs)

If the warnings setting has been set to True we can print a warning if the transient tip clearance is negative i.e. the rotor is rubbing. This is done rather than being included in the above checks as it would add a significant amount of complexity to them. We can also plot the transient response here, which is purely for demonstration. The first 5 seconds shows the machined clearance, followed by a 5 second spin up (not very realistic), the 5 seconds at cold rotating before heating starts, essentially a step change in gas temperature from 293K to 950K. 

In [ ]:
    if warm_rot_g < 0:
        if print_warnings:
            print(('WARNING: STAGE {}: POSSIBLE ROTOR-STATOR CONTACT IN '
                   'TRANSIENT HEATING'.format(stage_n)))
    if stage_n in plotting and transient_analysis:
        t_cold = [0, 5]
        t_spin = [5, 10]
        t_cold_rot = [10, 15]
        c_cold = [cold_stat_g*1000, cold_stat_g*1000]
        c_spin = [cold_stat_g*1000, cold_rot_g*1000]
        c_cold_rot = [cold_rot_g*1000, cold_rot_g*1000]
        t_trans = [i+t_cold_rot[-1] for i in t_trans]
        g_trans = [i*1000 for i in g_trans]
        t_hot = [max(t_trans), max(t_trans)+10]
        c_hot = [g_trans[-1], g_trans[-1]]
        plt.plot(t_cold, c_cold, 'tab:blue', label='Cold-static')
        plt.plot(t_spin, c_spin, color='black', linestyle='--')
        plt.plot(t_cold_rot, c_cold_rot, 'tab:green', label='Cold-rotating')
        plt.plot(t_trans, g_trans, 'tab:orange', label='Warm transient')
        plt.plot(t_hot, c_hot, 'tab:red', label='Hot soaked')
        plt.legend(prop={'size': 18})
        plt.xlabel('Time (s)', fontsize=20)
        plt.ylabel('Change in clearance (mm)', fontsize=20)
        plt.tick_params(axis="x", labelsize=20)
        plt.tick_params(axis="y", labelsize=20)

Finally for the stage function we package up all relevant dimensions and output them.

In [ ]:
    expansion_lims = [cold_stat_g, cold_rot_g, warm_rot_g, hot_rot_g,
                      thermal_calc[0], thermal_calc[1]]
    # List of the relevant dimensions
    dimensions = [r, H1, H2, H3, Cx_st, Cx_ro, ptc_st, ptc_ro, Ro_st,
                  Ri_ro, stator_blades_N, rotor_blades_N]
    return [To3, Po3, eff, mass, volume, work, length, dimensions, loss,
            loss_array, a3, Fx, Res, expansion_lims]

### Vortex functions

This section was going to include more functions for other vortex designs, however in the end 3D design was limited so for now only the free vortex is included. These angles are only used for input to meshing for CFD, they aren't used anywhere else in the code. Also, as the helium turbine tends to have very high hub-to-tip ratios (>0.9), the difference between the hub and tip blade angles in a free vortex is very small. 

In [ ]:
###################
# VORTEX FUNCTIONS#
###################


def free_vortex(angs, size, phi):
    """Return the hub, midspan and tip metal angles for a free vortex stage"""

    # Pull angles out of the array, converting them to radians
    a1, a2, b2, _, b3 = np.radians(angs)
    # Pull out radius and spans
    r, hst, hro = size
    # Free vortex has constant Vx so keeping rVt constant is simple
    a1h = np.arctan(r*np.tan(a1)/(r-hst/2))
    a1t = np.arctan(r*np.tan(a1)/(r+hst/2))
    a2h = np.arctan(r*np.tan(a2)/(r-hst/2))
    a2t = np.arctan(r*np.tan(a2)/(r+hst/2))
    # New relative flow angles depend on the flow coefficient at the span
    b2h = np.arctan(np.tan(a2h)-(r-hro/2)/(phi*r))
    b2t = np.arctan(np.tan(a2t)-(r+hro/2)/(phi*r))
    b3h = np.arctan(np.tan(a1h)-(r-hro/2)/(phi*r))
    b3t = np.arctan(np.tan(a1t)-(r+hro/2)/(phi*r))

    return [np.degrees([a1h, a1, a1t]), np.degrees([a2h, a2, a2t]),
            np.degrees([b2h, b2, b2t]), np.degrees([b3h, b3, b3t])]

### Loss functions

This section includes all of the functions associated with calculating losses in a stage. The first one is just a packaging up of the equation for finding stagnation pressure given a loss coefficient using the compressible gas relation:

\begin{align}
Y_p &= \frac{P_{01}-P_{02}}{P_{02}-P_{2}}\\
&
\end{align}
\begin{align}
P_{2} &= P_{02}\left(1+\frac{\gamma-1}{2}M^2\right)^{-\frac{\gamma}{\gamma-1}}\\
&= P_{02}F\\
&
\end{align}
\begin{align}
P_{01} &= Y_p(P_{02}-P_{2})+P_{02} \\
&= (Y_p(1-F)+1)P_{02} \\
& \\
P_{02} &= \frac{P_{01}}{(Y_p(1-F)+1)}
\end{align}

In [ ]:
##################
# LOSS FUNCTIONS #
##################


def Y_po(po1, Y, V, c, gamma):
    """Return the exit stagnation pressure after loss"""

    M = V/c  # Flow Mach number
    fac = (1+(gamma-1)*(M**2)/2)**(-gamma/(gamma-1))  # Makes next line neater
    # Find new stagnation pressure
    po2 = po1/(Y+1-Y*fac)

    return po2

The next function doesn't do anything particularly special, it just groups all of the loss calculations together. First all of the inputs are unpacked and shroud (TC) loss is found, as well as the fraction of flow leaking over the tip.

In [ ]:
def losses(angs, vels, states, dimensions, Res):
    """Return the entropy rises for given inputs in Stage"""

    # Create values from input arrays
    a1, a2, b2, b3 = angs
    _, V2, _, W3 = vels
    _, T2, T3, *_ = states
    t, g, _, H_st, H_ro, Cx_st, Cx_ro, w_st, w_ro = dimensions
    Re_st, Re_ro = Res
    # Calculate tip clearance effect first so that mass flow can be
    # accounted for in profile and trailing edge loss
    TC_calcs_st = TC(g, H_st, a1, a2, V2, T2)
    TC_st = TC_calcs_st[0]  # Entropy rise
    m_st = TC_calcs_st[1]  # Stator leakage mass flow fraction
    TC_calcs_ro = TC(g, H_ro, b2, b3, W3, T3)
    TC_ro = TC_calcs_ro[0]
    m_ro = TC_calcs_ro[1]
    TC_loss = TC_st+TC_ro  # Overall tip leakage entropy rise
    m_ls = [m_st, m_ro]  # Leakage mass flow fraction for use in analysis

Profile loss is calculated, and the profile loss coefficient is used in the trailing edge loss calculation. Both of these losses are scaled by the leakage fraction to reflect the fact that some of the total flow doesn't experience these losses. 

In [ ]:
    # Profile loss calculations
    profile_calcs_st = profile(Re_st, a1, a2, V2, T2)
    profile_calcs_ro = profile(Re_ro, b2, b3, W3, T3)
    profile_st = profile_calcs_st[0]*(1-m_st)
    profile_ro = profile_calcs_ro[0]*(1-m_ro)
    profile_loss = profile_st+profile_ro
    # Find the profile loss coefficients for the
    # momentum boundary layer thickness
    zeta_st = profile_calcs_st[1]
    zeta_ro = profile_calcs_ro[1]
    # Trailing edge loss calculations
    TE_st = TE(t, w_st, zeta_st, V2, T2)*(1-m_st)
    TE_ro = TE(t, w_ro, zeta_ro, W3, T3)*(1-m_ro)
    TE_loss = TE_st+TE_ro

Finally secondary loss is calculated and the outputs are given as the total loss, the loss split between rotor and stator, and as the breakdown between mechanisms.

In [ ]:
    # Secondary flow loss calculations
    secondary_st = secondary(a1, a2, Cx_st, H_st, V2, T2)
    secondary_ro = secondary(b2, b3, Cx_ro, H_ro, W3, T3)
    secondary_loss = secondary_st+secondary_ro
    # Add up entropy rises for the stator and rotor and find the overall loss
    loss_st = profile_st+TE_st+secondary_st+1.0*TC_st
    loss_ro = profile_ro+TE_ro+secondary_ro+TC_ro
    loss = loss_st+loss_ro
    loss_comp = [profile_loss, TE_loss,
                 secondary_loss, TC_loss]  # Array of loss components

    return loss, loss_st, loss_ro, loss_comp, m_ls

The profile, trailing edge and shroud loss correlations are all taken from [Loss Mechanisms in Turbomachines (Denton 1993)](https://asmedigitalcollection.asme.org/turbomachinery/article-abstract/115/4/621/434211). The profile loss uses a dissipation coefficient that scales with $Re^{-0.2}$.

In [ ]:
def profile(Re, a1, a2, V, T):
    """Return the profile entropy rise for the stage"""

    # Denton 1993
    v_frac = 1/np.sqrt(3)  # Fix the idealised velocity fraction
    # Calculate dissipation coefficient using correlation
    Cd = 0.002*(Re/500000)**(-0.2)
    # Calculate entropy loss coefficient for use in trailing edge loss function
    zeta = Cd*(2/v_frac+6*v_frac)*abs(np.tan(a2)-np.tan(a1))
    # Calculate entropy rise
    entropy = (zeta*0.5*V**2)/T

    return entropy, zeta

The trailing edge loss assumes a base pressure coefficient of -0.15 and takes the momentim boundary layer thickness to be 

\begin{equation}
\theta = \frac{\zeta_{profile}w}{2}
\end{equation}

where $w$ is the throat width. 

In [ ]:
def TE(t, w, zeta_profile, V, T):
    """Return the trailing edge entropy rise for the stage"""

    # Denton 1993
    SF = 1.4  # Fix the shape function for a turbulent boundary layer
    Cpb = -0.15  # Fix the base pressure coefficient
    # Calculate the momentum boundary layer thickness
    # using profile loss coefficient
    BL_mom = zeta_profile*w/2
    # Calculate the displacement boundary layer thickness
    BL_dis = SF*BL_mom
    # Calculate the entropy loss coefficient
    zeta = ((t+BL_dis)/w)**2-Cpb*t/w
    # Calculate the entropy rise
    entropy = (zeta*0.5*V**2)/T

    return entropy

The correlation for secondary loss is from [Improvements to the Ainley-Mathieson Method of Turbine Performance Prediction (Dunham & Came 1970)](https://asmedigitalcollection.asme.org/gasturbinespower/article-abstract/92/3/252/404009), adjusted by Denton (see CUED turbomachinery course) to fit data. 

In [ ]:
def secondary(a1, a2, Cx, H, V, T):
    """Return the secondary flow entropy rise for the stage"""

    # Dunham & Came 1970
    # Calculate the vector mean air angle
    am = np.arctan(0.5*(np.tan(a1)+np.tan(a2)))
    # Calculate the stagnation pressure loss coefficient
    Y = 0.375*0.1336*(Cx/H)*(np.cos(a2)**3) *\
        ((np.tan(a1)-np.tan(a2))**2)/(np.sqrt(np.cos(a1))*np.cos(am))
    # Convert to entropy rise assuming incompressible flow
    entropy = (Y*0.5*V**2)/T

    return entropy

The leakage loss assumes a shrouded turbine.

In [ ]:
def TC(g, H, a1, a2, V2, T):
    """Return the tip leakage entropy rise for the stage"""

    # Denton 1993
    Cc = 0.6  # Fix the shroud contraction coefficient
    # Calculate the leakage mass flow fraction
    m = g*Cc*np.sqrt(abs((1/np.cos(a2))**2-np.tan(a1)**2))/H
    # Calculate the entropy rise
    entropy = m*(V2**2)*(1-np.tan(a1)*(np.sin(a2)**2)/np.tan(a2))/T

    return entropy, m

### Gas properties

Functions have been written to calculate how thermodynamic and transport properties of helium and combustion products vary with Turbine emperature and pressure. As a monatomic gas helium is nearly always a perfect gas and so $c_p$ and $\gamma$ are constant. Properties for helium are taken from [Petersen](https://backend.orbit.dtu.dk/ws/portalfiles/portal/52768509/ris_224.pdf) and the equations for A1 jet fuel products have have been fit to data from [NASA](https://ntrs.nasa.gov/archive/nasa/casi.ntrs.nasa.gov/19690028554.pdf).

In [ ]:
##################
# GAS PROPERTIES #
##################


def thermo_props(T, gas):
    """Calculate gamma, cp and R for the gas"""

    # Helium
    if gas == 'He':
        gamma = 1.6625
        cp = 5187
        R = cp-cp/gamma
    # Combustion gases with A1 jet fuel (AFR = 50)
    if gas == 'A1':
        gamma = 1.41-8.49*10**(-5)*T
        cp = 0.22*T+951
        R = cp-cp/gamma

    return gamma, cp, R


def viscosity(T, gas):
    """Calculate viscosity of the gas"""

    # Helium
    if gas == 'He':
        mu = (3.674*10**(-7))*(T**0.7)
    # Combustion gases with A1 jet fuel (AFR = 50)
    if gas == 'A1':
        mu = (1.31*10**(-5)+0.0059*T-1.71*10**(-6)*T**2)*10**(-5)

    return mu


def Prandtl(T, P, gas):
    """Calculate Prandtl number of the gas"""

    # Helium
    if gas == 'He':
        Pr = 0.6728/(1+P*2.7*10**(-9))*(T/273)**(-(0.01-P*1.42*10**(-9)))
    # Combustion gases with A1 jet fuel (AFR = 50)
    if gas == 'A1':
        Pr = 0.716-7.66*10**(-6)*T

    return Pr


def conductivity(T, gas):
    """Calculate thermal conductivity of the gas"""

    # Helium
    if gas == 'He':
        k = 0.14789*(T/273)**0.6958
    # Combustion gases with A1 jet fuel (AFR = 50)
    if gas == 'A1':
        k = (0.0059*T+0.94)*10**(-2)

    return k

### Geometry

The first three of these functions calculate angles for either non-repeating or repeating stages. The equation for $\alpha_3$ for non-repeating stages can be derived from first principles using Euler's work equation and the definition of $\phi, \psi$ and $\Lambda$.

In [ ]:
######################
# GEOMETRY FUNCTIONS #
######################


def angles(phi, psi, Lambda, a1):
    """Return the flow angles for non-repeating stages"""

    # Define coefficients in the quadratic for tan(a3) to keep it clear
    a = Lambda*phi**2
    b = 2*psi*phi
    c = 2*psi*Lambda-Lambda*(phi*np.tan(np.radians(a1)))**2-2*psi+psi**2
    # Calculate angles, using positive root for a3
    a3 = np.arctan((-b+np.sqrt(b**2-4*a*c))/(2*a))
    a2 = np.arctan(np.tan(a3)+psi/phi)
    b2 = np.arctan(np.tan(a2)-1/phi)
    b3 = np.arctan(np.tan(a3)-1/phi)

    return a1, a2, b2, a3, b3


def repeating(phi, psi, Lambda, a1):
    """Return the flow angles for specified loading and repeating stage"""

    # Calculate angles
    a1 = np.arctan((1-0.5*psi-Lambda)/phi)
    a2 = np.arctan(np.tan(a1)+psi/phi)
    b2 = np.arctan(np.tan(a2)-1/phi)
    b3 = np.arctan(np.tan(a1)-1/phi)
    a3 = a1

    return a1, a2, b2, a3, b3


def velocities(angs, Vx):
    """ Return the velocities for the specified angles"""

    # Extract angles from array
    a1, a2, b2, a3, b3 = angs
    # Calculate velocities
    V1 = Vx/np.cos(a1)
    V2 = Vx/np.cos(a2)
    W2 = Vx/np.cos(b2)
    V3 = Vx/np.cos(a3)
    W3 = Vx/np.cos(b3)

    return V1, V2, W2, V3, W3

This function finds the throat width. Initially the Zweifel condition was used to find a pitch-to-chord ratio, however this proved to give far too large ratios (>1.5), so you can also define a $p/C_x$ and use that to find pitch and throat width. 

In [ ]:
def p_w(Cx, a1, a2, ptoC):
    """Return the pitch and throat width for the stage"""
    # If pitch to chord is specified, use it
    if ptoC > 0:
        ptc = ptoC
        p = Cx*ptc
    # Otherwise use Zweifel criterion
    else:
        Z = 0.8  # Fix the Zweifel coefficient
        # Calculate pitch to chord ratio
        ptc = 0.5*Z/((np.cos(a2)**2)*abs(np.tan(a2)-np.tan(a1)))
        # Calculate pitch
        p = Cx*ptc
    # Calculate throat width
    w = p*np.cos(a2)

    return w, ptc

### Thermo-mechanical modelling

The aim of the thermo-mechanical modelling is to get an estimate for the mass of the turbine based on stress limits, which could then be used in an optimisation between efficiency and mass, and also to investigate the behaviour of the turbine as it expands during start up. The properties of the metal used for the drum and case will vary with temperature, and it's important that stress limits aren't breached at any point during operation, so functions have been implemented to calculate the material properties given a temperature.

Steel was considered, although it later became clear that it is entirely unsuitable for such high temperature applications. Equations are taken from [Eurocode 3](https://www.gaprojekt.com/sites/default/files/legislation/Eurocode%203%20-Design%20of%20steel%20structures%20-%20Part%201-2%20-%20en.1993.1.2.2005.pdf) and [Effect of Temperature on Strength and Elastic Modulus of High-Strength Steel (Wang, Liu & Kodur 2012](https://ascelibrary.org/doi/full/10.1061/%28ASCE%29MT.1943-5533.0000600?casa_token=gEin4Bf-LIoAAAAA%3APrZ4a2cRjiZK_evQdKQ3_Kbe6G5cxV_eDWvCJ9mB1KqHtvtvXhA5DyCRmdTF1IZv59mpKFo4wGE). 

In [ ]:
######################################
# MASS, THERMAL AND STRESS FUNCTIONS #
######################################


def steel(T):
    """Return the properties of steel at temperature T"""

    # Poission's ratio (Eurocode 3)
    nu_m = 0.3
    # Density (Eurocode 3)
    rho_m = 7850
    # Thermal strain (Eurocode 3)
    strain = 1.2*10**(-5)*(T-273)+4*10**(-9)*(T-273)**2-2.416*10**(-4)
    # Specific heat capacity (Eurocode 3)
    if T < 873:
        c_m = 425+7.73*10**(-1)*(T-273) -\
            1.69*10**(-3)*(T-273)**2+2.22*10**(-6)*(T-273)**3
    else:
        c_m = 666+13002/(1011-T)
    # Thermal conductivity (Eurocode 3)
    k_m = 54-3.33*(10**(-2)*(T-273))
    # Yield stress (Wang, Liu & Kodur 2013)
    if T < 723:
        sigma_m = 500*10**6
    else:
        sigma_m = (4.32*np.exp(-(T-273)/880)-1.6)*500*10**6
    # Young's modulus
    E_m = (1.02-0.035*np.exp((T-273)/280))*210*10**9

    return nu_m, rho_m, strain, c_m, k_m, sigma_m, E_m

Inconel 718 is commonly used in turbine components due to it's excellent properties at high temperatures. The equations for properties used here are from [Temperature Measurement
and Numerical Prediction in Machining Inconel 718 (Díaz-Álvarez et al. 2017)](https://www.mdpi.com/1424-8220/17/7/1531), [High temperature deformation behavior of solid and semi-solid alloy 718 (Lewandowski & Overfelt 1999)](https://www.sciencedirect.com/science/article/pii/S1359645499002529?casa_token=9ThKD-SWwtYAAAAA:Lw_9mdJGymEDf-rEB9ZysgdD_V8qgaYpJ1ggpxkf16sayPVcKJKNqEC4C0JPOUke0xeDWzV4jtQ), [Innovative use of multi-heat sources for improvement of tool life in thermally assisted machining of high-strength material (Woo & Lee 2019)](https://www.sciencedirect.com/science/article/pii/S152661251830848X?casa_token=mefx7gEqOt0AAAAA:Pm9-wgSmRTVz1KE6_okhvWGc_Bxyt2JzOTvlLWUjME-QZsXETiDJzM3QrkM5XG7cZvfVH4ppWD8) and [High temperature deformation of Inconel 718 (Thomas at el. 2006)](https://www.sciencedirect.com/science/article/pii/S0924013606004985?casa_token=7CXY19az25cAAAAA:wZ-ej3-qFwfebQKcNMs5QCQbsyhRzTGh3wVPvWpYIJYFh8DHvAfGxEj6aCXcq7mpbzWd6kgOILQ).

In [ ]:
def inconel(T):
    """Return the properties of Inconel 718 at temperature T"""

    # Poission's ratio (Díaz-Álvarez et al. 2017)
    nu_m = 0.3
    # Density (Díaz-Álvarez et al. 2017)
    rho_m = 8300
    # Thermal strain (Lewandowski & Overfelt 1999)
    strain = integrate.quad(lambda x: (1.28+5*10**(-4)*(x-366))*10**(-5),
                            293, T)
    # Specific heat capacity (Díaz-Álvarez et al. 2017)
    c_m = 400+0.15*np.exp((T-300)/90)
    # Thermal conductivity (Díaz-Álvarez et al. 2017)
    k_m = (1/60)*T+5
    # Yield stress (Woo & Lee 2019)
    sigma_m = (1250-np.exp((T-273)/128))*10**6
    # Young's modulus (Thomas et el. 2006)
    E_m = 2*(1+nu_m)*(1-0.5*(T-300)/1673)*8.31*10**10

    return nu_m, rho_m, strain[0], c_m, k_m, sigma_m, E_m

The mass of the blades in a blade row is found by first calculating the total number of blades using $p/C_x$ and then multiplying by the mass of a single blade, found using the blade section area. The mass of the shroud, which is adds significant stress to the rotor, is found by assuking the shroud to be a 1mm thick plate at the tip of each blade. 

In [ ]:
def blade_mass(blade_sizes, rho_m, row_type, shroud=True):
    """Return the mass of the blades in the blade row"""

    rm, ptc, H, Cx, A = blade_sizes
    # Calculate the total number of blades
    N = round(2*np.pi*rm/(ptc*Cx))
    # Calculate blade mass
    m = rho_m*H*A*N
    # Include shroud mass if specified
    m_shroud = 0
    if shroud:
        # Shroud thickness of 1.5mm
        t_shroud = 0.001
        # Calculation different on rotor or stator
        if row_type == 'rotor':
            # Single blade shroud mass
            m_shroud = rho_m*Cx*t_shroud*2*np.pi*(rm+H/2)/N
        else:
            # Single blade shroud mass
            m_shroud = rho_m*Cx*t_shroud*2*np.pi*(rm-H/2)/N
        # Add to total
        m += m_shroud

    return m, N, m_shroud

This function checks if the stress at the root of a rotor blade exceeds the yield stress of the material, and returns the stress for use with the drum. The equation for root stress is simple enough to derive, see Fluid mechanics and thermodynamics of turbomachinery by Dixon and Hall for details.

In [ ]:
def blade_stress(rm, H, omega, Cx, N, rho_m, A, m_shroud, sigma_y, stage_n):
    """Return the blade root stress"""

    # Find the hub and rip radii
    rt = rm+H/2
    rh = rm-H/2
    # Find the stress
    sigma_root = 0.5*rho_m*rt**2*omega**2*(1-(rh/rt)**2)+m_shroud*rt*omega**2/A
    # Warn if root stress exceeds yield
    if sigma_root > sigma_y:
        global print_warnings
        if print_warnings:
            print('WARNING: STAGE {}: MATERIAL LIMITS '.format(stage_n),
                  'EXCEEDED AT BLADE ROOT')
    # Find the apparent pressure on the rotor ring from the blades
    P_blades = N*sigma_root*A/(2*np.pi*rh*1.5*Cx)

    return P_blades

The drum (rotor) mass is determined by what the inner radius must be to withstand the combined loadsings due to rotation, external pressure and the centrifugal force of the blades. The blade force is treated as a unform 'negative pressure'. There are two extreme cases that should be satsified: when the turbine is pressurised but not rotating, and when it is rotating but not pressurised. The derivation for the inner radius is algebra heavy, but can be done by combining formulas found in Roark's Formulas for Stress and Strain. 

In [ ]:
def rotor_mass(rm, H_ro, H_st, Cx_ro, Cx_st, sigma_y, Po1, P_blades,
               omega, rho_m, nu, stage_n, n_stages):
    """Return the mass of the rotor ring"""

    global print_warnings
    # Find the outer radius of the hub
    Ro = rm-H_st/2
    # First find the rotating condition with no external pressure
    a = rho_m*omega**2*(1-nu)
    b = 2*(rho_m*omega**2*Ro**2*(1+nu)-2*sigma_y)
    c = 4*sigma_y*Ro**2-8*P_blades*Ro**2-rho_m*omega**2*(3+nu)*Ro**4

The formula will break down if the stress limit is exceeded even when $R_i=0$, so check for these cases and print a warning if necessary.

In [ ]:
    # If the material is not strong enough then the equation breaks down
    # Set Ri to 0 and post a warning
    if b**2-4*a*c < 0:
        Ri_rot = 0
        if print_warnings:
            print('WARNING: STAGE {}: MATERIAL LIMITS '.format(stage_n),
                  'EXCEEDED IN HUB: BURST')
    elif (-b-np.sqrt(b**2-4*a*c))/(2*a) < 0:
        Ri_rot = 0
        if print_warnings:
            print('WARNING: STAGE {}: MATERIAL LIMITS '.format(stage_n),
                  'EXCEEDED IN HUB: BURST')

The first and last stages of the drum need to extend to the shaft, so the inner radius must always be zero. 

In [ ]:
    elif stage_n in (1, n_stages):
        Ri_rot = 0
    else:
        Ri_rot = np.sqrt((-b-np.sqrt(b**2-4*a*c))/(2*a))

Checking the static condition of just pressure, if this radius is less than that for rotating then it is the limiting case.

In [ ]:
    # Find the static condition with only external pressure.
    # Again check if material limits are reached
    if Ro**2*(sigma_y-2*Po1) < 0:
        Ri_P = 0
        if print_warnings:
            print('WARNING: STAGE {}: MATERIAL LIMITS '.format(stage_n),
                  'EXCEEDED HUB: PRESSURE')
    else:
        Ri_P = np.sqrt(Ro**2*(sigma_y-2*Po1)/sigma_y)
    # Take whichever inner radius is smaller
    if Ri_rot < Ri_P:
        Ri = Ri_rot
    else:
        Ri = Ri_P

Now use the thickness to calculate mass. For low stage numbers where the blade span increases significantly through a stage there's a possibility that the thickness calculated based on the inlet radius would intersect with the shaft at the exit, so this is accounted for here.

In [ ]:
    # Calculate a thickness
    thk = Ro-Ri
    # Calculate mass for separate possibilities
    if rm-H_ro/2-thk < 0:
        # Mean hub radius of stage
        R_m = (2*rm-(H_st+H_ro)/2)/2
        # Calculate the mass of the rotor ring,
        # which has to cover the rotor too
        m = rho_m*1.5*(Cx_ro+Cx_st)*np.pi*(R_m**2-(Ri/2)**2)
    else:
        # Mean hub radius of stage
        R_m = (2*rm-(H_st+H_ro)/2)/2-thk/2
        # Calculate the mass of the rotor ring,
        # which has to cover the rotor too
        m = rho_m*1.5*(Cx_ro+Cx_st)*2*np.pi*R_m*thk

    return m, Ri

The mass of the case (stator) is much more simple as there is no limit to the outer radius, it is simply as thick as it must be. The only check is if the internal pressure is high enough to cause yielding on the inner face. The equation used is for a thick walled cylinder found in Roark's Formulas for Stress and Strain.

In [ ]:
def stator_mass(rm, H_st, H_ro, Cx_st, Cx_ro, sigma_y, Po1, rho_m, stage_n):
    """Return the mass of the stator ring"""

    global print_warnings
    # Find the inner radius of the case
    Ri = rm+H_ro/2
    # Find the outer radius required to resist the pressure
    # Check for material failure first
    if 2*Po1 > sigma_y:
        if print_warnings:
            print('WARNING: STAGE {}: MATERIAL LIMITS '.format(stage_n),
                  'EXCEEDED IN CASE: PRESSURE')
        Ro = 1.1*Ri
    else:
        Ro = np.sqrt(sigma_y*Ri**2/(sigma_y-2*Po1))
    # Calculate thickness
    thk = Ro-Ri
    # Mean case radius of stage
    R_m = (2*rm+(H_st+H_ro)/2)/2+thk/2
    # Calculate the mass of the case, which has to cover the rotor too
    m = rho_m*1.5*(Cx_st+Cx_ro)*2*np.pi*R_m*thk

    return m, Ro

This stage_mass function is where the mass of all the components is summed to get the total stage mass. It is also where the expansions are calculated as the required values are convenient to keep enclosed here. We first unpack the relevant values and set saftey factors.

In [ ]:
def stage_mass(To1, Po1, Po3, dimensions, Omega, blade_areas, materials,
               stage_n, n_stages, trans=[False, False]):
    """Return the total mass of the stage"""

    global print_warnings
    # Extract values from list
    _, _, r, H_st, H_ro, Cx_st, Cx_ro, _, _, ptc_st, ptc_ro = dimensions
    A_st, A_ro = blade_areas
    # Set the materials for the rotor and stator
    stator_mat, rotor_mat = materials
    # Set safety factors
    SF_st = 1.0
    SF_ro = 1.0

Dealing with the stator first, find the material properties and warn if they've become unusabel at the temperature.

In [ ]:
    # Find the properties for the stator material:
    if stator_mat == 'inconel':
        nu_m, rho_m, _, _, _, sigma_m, E_m = inconel(To1)
    if stator_mat == 'steel':
        nu_m, rho_m, _, _, _, sigma_m, E_m = steel(To1)
    if sigma_m <= 0 or E_m <= 0:
        if print_warnings:
            print('WARNING: STAGE {}: MATERIAL THERMAL '.format(stage_n),
                  'LIMITS REACHED')

Call each of the functions and sum masses. 

In [ ]:
    # Find the mass and number of stator blades
    stator_blade_sizes = [r, ptc_st, H_st, Cx_st, A_st]
    stator_blade_calc = blade_mass(stator_blade_sizes, rho_m, 'stator')
    stator_blade_mass = stator_blade_calc[0]
    stator_blade_N = stator_blade_calc[1]
    # Find the mass of the stator case and sum
    stator_case_calc = stator_mass(r, H_st, H_ro, Cx_st, Cx_ro,
                                   sigma_m/SF_st, Po1, rho_m, stage_n)
    stator_case_mass = stator_case_calc[0]
    stator_Ro = stator_case_calc[1]
    stator_total_mass = stator_case_mass+stator_blade_mass

The same procedure is repeated for the rotor. This time we also need to find the stress on the blades.

In [ ]:
    # Find the properties for the rotor material:
    if rotor_mat != stator_mat:
        if rotor_mat == 'inconel':
            nu_m, rho_m, _, _, _, sigma_m, E_m = inconel(To1)
        if rotor_mat == 'steel':
            nu_m, rho_m, _, _, _, sigma_m, E_m = steel(To1)
    if sigma_m <= 0 or E_m <= 0:
        if print_warnings:
            print('WARNING: STAGE {}: MATERIAL THERMAL '.format(stage_n),
                  'LIMITS REACHED')
    # Find the mass and number of rotor blades
    rotor_blade_sizes = [r, ptc_ro, H_ro, Cx_ro, A_ro]
    rotor_blade_calc = blade_mass(rotor_blade_sizes, rho_m, 'rotor')
    rotor_blade_mass = rotor_blade_calc[0]
    rotor_blade_N = rotor_blade_calc[1]
    m_shroud = rotor_blade_calc[2]
    # Find the stress from the rotor blade roots
    P_blades = blade_stress(r, H_ro, Omega, Cx_ro, rotor_blade_N, rho_m,
                            A_ro, m_shroud, sigma_m/SF_ro, stage_n)
    # Find the mass of the rotor ring
    rotor_ring_calc = rotor_mass(r, H_ro, H_st, Cx_ro, Cx_st, sigma_m/SF_ro,
                                 Po1, P_blades, Omega, rho_m, nu_m,
                                 stage_n, n_stages)
    rotor_ring_mass = rotor_ring_calc[0]
    rotor_Ri = rotor_ring_calc[1]
    rotor_total_mass = rotor_ring_mass+rotor_blade_mass

Sum up the masses for the total stage mass. Find the elongation, first due to just rotation and then due to rotation and heating.

In [ ]:
    # Sum for total mass and blades
    stage_m = rotor_total_mass+stator_total_mass
    # Changes in radii from cold-static to cold-rotating
    stator_Rs = [r+H_st/2, stator_Ro]
    rotor_Rs = [rotor_Ri, r-H_ro/2]
    dr_cold = elongation(materials, stator_Rs, rotor_Rs, m_shroud, A_ro,
                         Omega, Po1, Po3, P_blades, 293)
    # Changes in radii from cold-static to hot-rotating
    dr_hot = elongation(materials, stator_Rs, rotor_Rs, m_shroud, A_ro,
                        Omega, Po1, Po3, P_blades, To1)

Finally call the transient analysis function if it's required, and return.

In [ ]:
 # Perform transient analysis if asked for
    if trans[0]:
        taus = [trans[1], trans[2]]
        trans_ans = transient(materials, stator_Rs, rotor_Rs, m_shroud, A_ro,
                              Omega, Po1, Po3, P_blades, To1, taus)
        return trans_ans
    return [stage_m, stator_blade_N, rotor_blade_N, stator_Ro,
            rotor_Ri, dr_cold, dr_hot]

The function for blade force applies the SFME

In [ ]:
def blade_force(P1, P2, r, H1, H2):
    """Return the axial force on the blade row"""

    # SFME with constant axial velocity: just pressure difference
    Fx = 2*np.pi*r*(H2*P2-H1*P1+(H2-H1)*(P1+P2)/2)

    return Fx

Transient analysis applies the lumped heat capacity model to the hub and case over a period of 10 time constants.

In [ ]:
def transient(materials, stator_Rs, rotor_Rs, m_shroud,
              A_ro, omega, Po1, Po3, P_blades, T, taus):
    """Full transient analysis of the changing tip clearance"""

    # Time over which to calculate response
    t = np.linspace(0, 10*max(taus), 1000)
    tau_st, tau_ro = taus
    T_st = [(293-T)*np.exp(-i/tau_st)+T for i in t]
    T_ro = [(293-T)*np.exp(-i/tau_ro)+T for i in t]
    # Find changes in radii of stator and rotor over the time period
    dr_st = [elongation(materials, stator_Rs, rotor_Rs, m_shroud, A_ro,
                        omega, Po1, Po3, P_blades, i)[0] for i in T_st]
    dr_ro = [elongation(materials, stator_Rs, rotor_Rs, m_shroud, A_ro,
                        omega, Po1, Po3, P_blades, i)[1] for i in T_ro]
    # Find subsequent tip gaps
    g_trans = [dr_st[i]-dr_ro[i] for i, _ in enumerate(t)]

    return g_trans, t

Elongation from rotation, pressure and thermal expansion is found here. As with the mass function, unpack the inputs and find the material properties. 

In [ ]:
def elongation(materials, stator_Rs, rotor_Rs, m_shroud,
               A_ro, omega, Po1, Po3, P_blades, T):
    """Return the elongation of components"""

    # Unpack variables
    stator_mat, rotor_mat = materials
    Ri_st, Ro_st = stator_Rs
    Ri_ro, Ro_ro = rotor_Rs
    # Get material properties
    if stator_mat == 'inconel':
        nu_st, _, strain_st, _, _, _, E_st = inconel(T)
    if stator_mat == 'steel':
        nu_st, _, strain_st, _, _, _, E_st = steel(T)
    if rotor_mat == 'inconel':
        nu_ro, rho_ro, strain_ro, _, _, _, E_ro = inconel(T)
    if rotor_mat == 'steel':
        nu_ro, rho_ro, strain_ro, _, _, _, E_ro = steel(T)

The elongation is a combination of mechanical strain, found using forumals from Roark and a similar analysis to the blade root stress, and the thermal strain, which is found by integrating with a thermal expansion coefficient in the materials functions. 

In [ ]:
    # Stator
    # Increase in inner radius due to pressure
    dr_P_st = Po1*Ri_st/E_st*((Ro_st**2+Ri_st**2)/(Ro_st**2-Ri_st**2)+nu_st)
    # Thermal expansion
    dr_T_st = strain_st*Ri_st
    # Total
    dr_st = dr_P_st+dr_T_st
    # Rotor
    # Increase in outer radius due to rotation
    dr_r_ro = rho_ro*omega**2*Ro_ro/(4*E_ro) *\
        ((1-nu_ro)*Ro_ro**2+(3+nu_ro)*Ri_ro**2)
    # Increase in outer radius due to combined blade and gas pressure
    dr_P_ro = (P_blades+Po3-Po1)*Ro_ro/E_st *\
        ((Ro_ro**2+3*Ri_ro**2)/(Ro_ro**2-Ri_ro**2)-nu_ro)
    # Increase in blade height from blade mass
    dH_blade = rho_ro*Ri_st**2*omega**2/(2*E_ro) *\
        (2*Ri_st/3-Ro_ro+Ro_ro**3/(3*Ri_st**2))
    # Increase in blade height from shroud mass
    dH_shroud = m_shroud*Ri_st*omega**2/(E_ro*A_ro)*(Ri_st-Ro_ro)
    # Thermal expansion (outer radius being the tip of the blade)
    dr_T_ro = strain_ro*Ri_st
    # Total
    dr_ro = dr_r_ro+dr_P_ro+dr_T_ro+dH_blade+dH_shroud

    return dr_st, dr_ro

The thermal function finds heat transfer characteristics of the stage. First unpack inputs.

In [ ]:
def thermal(vels, states, pressures, dimensions, materials, gas):
    """Return thermal behaviour of the stage at start up"""

    # Extract stage parameters
    V1, V2, W2, W3, V3 = vels
    T1, T2, T3, rho1, rho2, rho3, mu2, mu3, _ = states
    P1, P2, P3 = pressures
    _, _, r, H_st, H_ro, Cx_st, Cx_ro, _, _, _, _, Ro_st, Ri_ro = dimensions
    stator_mat, rotor_mat = materials

To find a heat transfer coefficient we use the correlation for a flat plate with a turbulent boundary layer. This requires Reynolds and Prandtl numbers, which are takeen to be the avearge over a stage. The conductivity of the gas is also found.

In [ ]:
    # Biot number calculation
    # First need a characteristic length s=V/A
    s_st = (Ro_st**2-(r+H_st/2)**2)/(2*(r+H_st/2))
    s_ro = ((r-H_ro/2)**2-Ri_ro**2)/(2*(r-H_ro/2))
    # Find the heat transfer coefficient for a turbulent boundary layer
    # Need average properties across the stage
    mu1 = viscosity(T1, gas)
    Re_av_st = 1.5*Cx_st*(rho1*V1/mu1+rho2*V2/mu2)/2 +\
        1.5*Cx_ro*(rho2*V2/mu2+rho3*V3/mu3)/2
    Re_av_ro = 1.5*Cx_st*(rho1*W3/mu1+rho2*W2/mu2)/2 +\
        1.5*Cx_ro*(rho2*W2/mu2+rho3*W3/mu3)/2
    Pr_av = (Prandtl(T1, P1, gas)+2*Prandtl(T2, P2, gas) +
             Prandtl(T3, P3, gas))/4
    k_gas_av = (conductivity(T1, gas)+2*conductivity(T2, gas) +
                conductivity(T3, gas))/4

These are applied to the correlation for Nusselt number, which is then used to find a heat transfer coefficient. A Biot number is found to determine the validity of the lumped capacity model (generally Bi>>0.1 so it techincally isn't valid but is useful for comparison). Finally the timeconstant for the lumped model is found and output. 

In [ ]:
    # Calculate Nusselt numbers
    Nu_st = 0.0296*Re_av_st**(4/5)*Pr_av**(1/3)
    Nu_ro = 0.0296*Re_av_ro**(4/5)*Pr_av**(1/3)
    # Calculate heat transfer coefficients
    h_st = Nu_st*k_gas_av/(1.5*(Cx_st+Cx_ro))
    h_ro = Nu_ro*k_gas_av/(1.5*(Cx_st+Cx_ro))
    # Material properties
    if stator_mat == 'inconel':
        _, rho_st, _, c_st, k_st, *_ = inconel(293)
    if stator_mat == 'steel':
        _, rho_st, _, c_st, k_st, *_ = steel(293)
    if rotor_mat == 'inconel':
        _, rho_ro, _, c_ro, k_ro, *_ = inconel(293)
    if rotor_mat == 'steel':
        _, rho_ro, _, c_ro, k_ro, *_ = steel(293)
    # Calculate the Biot numbers
    Bi_st = h_st*s_st/k_st
    Bi_ro = h_ro*s_ro/k_ro
    # Calculate lumped mass time constants. To account for different
    # conductivities, the time constant is scaled by the ratio of the rotor
    # and stator materials conductivities
    tau_st = (k_ro/k_st)*c_st*rho_st*s_st/h_st
    tau_ro = (k_st/k_ro)*c_ro*rho_ro*s_ro/h_ro

    return tau_st, tau_ro, Bi_st, Bi_ro

### Spline

This function just returns a spline for the turbine parameters. The spline is of the maximum order for the number of points given. 

In [ ]:
###########################
# INTERPOLATION FUNCTIONS #
###########################


def spline(n, y):
    """Return spline of loadings using specified control points"""

    # Array of stages
    stages = [0]*len(y)
    stages[0] = 1
    stages[-1] = n
    if len(stages) > 2:
        for i in range(1, len(stages)-1):
            stages[i] = 1+i*(n-1)/(len(stages)-1)
    order = len(stages)-1
    # Spline the points
    tck = sciint.splrep(stages, y, k=order)
    xnew = np.arange(1, n+1, 1)
    ynew = sciint.splev(xnew, tck)

    return ynew

### Optimisation

This function has packaged up a procedure for optimising a turbine with a given number of stages using two control points per variable. The input is an output from the turbine function, and the inputs to that are extracted. 

In [ ]:
#####################
# OPTIMISE FUNCTION #
#####################


def optimise(start_turbine):
    """Find an optimum design point from a given start"""

    # Extract starting inputs
    Po1, To1, mdot, Omega, W, t, g, phi, psi, Lambda, AR, dho,\
        n, ptc, ain, gas = start_turbine[11]
    # Define starting points
    phi0 = phi[0]
    psi0 = psi[0]
    Lam0 = Lambda[0]
    AR0 = AR[0]
    dho0 = dho[0]
    phi1 = phi[-1]
    psi1 = psi[-1]
    Lam1 = Lambda[-1]
    AR1 = AR[-1]
    dho1 = dho[-1]

Next, limits to each variable are defined and a function is made to be used in the optimiser that takes a single set of arguments. The efficiency is negative as it is a minimisation problem. 

In [ ]:
    # Define limits
    phi_lim = (0.1, 1.5)
    psi_lim = (0.4, 3.0)
    Lam_lim = (0, 1)
    AR_lim = (0.1, 5)
    dh_lim = (1, 5)

    def turbine_calcs(args):
        """Takes a list of arguments and feeds them to the turbine function"""
        # Extract arguments
        ph1, ph2, ps1, ps2, L1, L2, AR1, AR2, dh1, dh2 = args
        # Form them into lists
        phi = [ph1, ph2]
        psi = [ps1, ps2]
        Lambda = [L1, L2]
        dho = [dh1, dh2]
        AR = [AR1, AR2]
        # Pass them to the turbine to find efficiency
        eff = turbine(Po1, To1, mdot, Omega, W, t, g, phi, psi, Lambda, AR,
                      dho, n, ptc, ain, gas)[0]
        # Return the negative of efficiency to allow for minimising
        return -eff

These two functions define the constraint on the exit angles being less than 73º.

In [ ]:
    def constraint_a2(args):
        """Constraint on a2"""
        # Extract and form lists
        ph1, ph2, ps1, ps2, L1, L2, *_ = args
        phi = [ph1, ph2]
        psi = [ps1, ps2]
        Lambda = [L1, L2]
        # Spline over n
        phi = spline(n, phi)
        psi = spline(n, psi)
        Lambda = spline(n, Lambda)
        # Calculate angles
        a2_max = 0
        a1 = ain
        for ph, ps, La in zip(phi, psi, Lambda):
            ang_check = angles(ph, ps, La, a1)
            a2 = np.degrees(ang_check[1])
            a1 = ang_check[3]
            if abs(a2) > a2_max:
                a2_max = abs(a2)
        # Return a value that should be greater than zero
        return 73-a2_max

    def constraint_b3(args):
        """Constraint on b2"""
        # Extract and form lists
        ph1, ph2, ps1, ps2, L1, L2, *_ = args
        phi = [ph1, ph2]
        psi = [ps1, ps2]
        Lambda = [L1, L2]
        # Spline over n
        phi = spline(n, phi)
        psi = spline(n, psi)
        Lambda = spline(n, Lambda)
        # Calculate angles
        b3_max = 0
        a1 = ain
        for ph, ps, La in zip(phi, psi, Lambda):
            ang_check = angles(ph, ps, La, a1)
            b3 = np.degrees(ang_check[4])
            a1 = ang_check[3]
            if abs(b3) > b3_max:
                b3_max = abs(b3)
        # Return a value that should be greater than zero
        return 73-b3_max

A starting point is defined and the bounds and constraints specified in a format that the optimiser requires.

In [ ]:
    # Form the starting point list
    x0 = [phi0, phi1, psi0, psi1, Lam0, Lam1, AR0, AR1, dho0, dho1]
    # Form the tuple of bounds
    bnds = (phi_lim, phi_lim, psi_lim, psi_lim, Lam_lim, Lam_lim,
            AR_lim, AR_lim, dh_lim, dh_lim)
    # Form the tuple of constraints
    cons = cons = ({'type': 'ineq', 'fun': constraint_a2},
                   {'type': 'ineq', 'fun': constraint_b3})

Minimisation is done with the SLSQP (sequential least squares programming) algorithm. 

In [ ]:
    # Find the minimum
    res = minimize(turbine_calcs, x0, method='SLSQP',
                   bounds=bnds, constraints=cons)
    # Extract the optimal variable and return them
    phi1, phi2, psi1, psi2, Lam1, Lam2, AR1, AR2, dho1, dho2 = res['x']
    return [phi1, phi2], [psi1, psi2], [Lam1, Lam2], [AR1, AR2], [dho1, dho2]

# prof_gen.py

The prof_gen.py file contains code to generate blade profiles. Profiles are generated using Bezier polynomials, with a Vinokur distribution of points applied for meshing purposes. These functions are all from CJC and apply the procedure for profile generation. 

In [ ]:
"""Module contains functions generating blade profiles from CJC
and length, area and plotting functions from WFD"""

# Import required modules
import math
import csv
import numpy as np
import scipy.optimize as SciOpt
from scipy.integrate import simps


###############################
# PROFILE GENERATOR FUNCTIONS #
###############################


def binomial(i, n):
    """Binomial coefficient"""

    return math.factorial(n) / float(math.factorial(i) * math.factorial(n - i))


def bernstein(t, i, n):
    """Bernstein polynom"""

    return binomial(i, n) * (t ** i) * ((1 - t) ** (n - i))


def bezier(t, points):
    """Calculate coordinate of a point in the bezier curve"""

    n = len(points) - 1
    x = y = 0
    for i, pos in enumerate(points):
        bern = bernstein(t, i, n)
        x += pos[0] * bern
        y += pos[1] * bern

    return x, y


def bezier_curve_range(n, points):
    """Range of points in a curve bezier"""

    for i in range(n):
        t = i / float(n - 1)
        yield bezier(t, points)


def GEN_TURNING(Xi1, Xi2, controls):
    """Generate x,y of a bezier from control points"""

    spacings = np.linspace(0, 1, len(controls)+2)[1:-1]
    CPs = []
    CPs.append([0.0, 0.0])
    for i, _ in enumerate(controls):
        CPs.append([spacings[i]+min(spacings[i], 1.-spacings[i])*controls[i],
                    spacings[i]-min(spacings[i], 1.-spacings[i])*controls[i]])
    CPs.append([1.0, 1.0])

    steps = 100
    x = []
    y = []
    for point in bezier_curve_range(steps, CPs):
        x.append(point[0])
        y.append(point[1])

    # Integrate profile
    xcam = [0.]
    ycam = [0.]

    DXi = Xi2-Xi1
    for i in range(1, len(x)):
        Xi = Xi1+DXi*(y[i]+y[i-1])*0.5
        xcam.append(xcam[-1]+(x[i]-x[i-1])*np.cos(Xi))
        ycam.append(ycam[-1]+(x[i]-x[i-1])*np.sin(Xi))

    DX = xcam[-1]
    ycam = ycam/DX
    xcam = xcam/DX

    return(np.array(xcam), np.array(ycam))


def GEN_BZ_2D(controls_x, controls_y):
    """Generate x,y of a bezier from control points"""

    CPs = []
    for i, _ in enumerate(controls_x):
        CPs.append([controls_x[i], controls_y[i]])

    steps = 100
    controlPoints = CPs
    x = []
    y = []
    for point in bezier_curve_range(steps, controlPoints):
        x.append(point[0])
        y.append(point[1])

    return(np.array(x), np.array(y))


def dist_vino(n, xst, xen, s1, s2):
    """Vinokur distribution"""

    # INPUT DESCRIPTION #
    # n - number of points
    # xst - xstart
    # xen - xend
    # s1 - start spacing
    # s2 - end spacing

    s1 = s1/(xen - xst)
    s2 = s2/(xen - xst)
    eta = np.arange(n)
    a = math.sqrt(s2)/math.sqrt(s1)
    b = 1.0/((n-1)*math.sqrt(s1*s2))
    nm1 = float(n-1)

    def trans(delta):
        return math.sinh(delta)/delta - b

    left = 0.00001
    delta = SciOpt.brentq(trans, left, 100)
    u = 0.5*(1.0 + np.tanh(delta*(eta/nm1 - 0.5))/(np.tanh(delta*0.5)))
    s = u/(a + (1-a)*u)

    return xst + (xen-xst)*s


def F_CF(psi):
    """Function to generate class function of psi"""

    C = (psi**0.5)*(1.0-psi)
    return C


def F_TF_bez(controls_cam_x, controls_cam_y, TE, Oval_frac):
    """Function that returns thickness contributions due to parameters"""

    psi = dist_vino(200, 0, 1, 1./2000, 1./500)
    X, S = GEN_BZ_2D(controls_cam_x, controls_cam_y)
    S = np.interp(psi, X, S)

    S = S+Oval_frac*S[0]*(1.-psi)**18
    C = F_CF(psi)  # Calculate class function

    TF = S*C+psi*TE/2  # Combine to form thickness funtion

    return TF


def calc_norm(x_cam, y_cam):
    """Calculate the normal to the camber line"""

    s = np.zeros((len(x_cam)))  # Initialise camber length array
    grad = np.zeros((len(x_cam), 2))  # Initialise camber gradient array
    norm = np.zeros((len(x_cam), 2))  # Initialise camber normals array

    for i in range(1, len(x_cam)):
        # Calculate cumsum length of camber
        s[i] = s[i-1]+((x_cam[i]-x_cam[i-1])**2+(y_cam[i]-y_cam[i-1])**2)**0.5

    grad[:-1, 0] = (x_cam[1:]-x_cam[:-1])/(s[1:]-s[:-1])  # Calculate dx/ds
    grad[:-1, 1] = (y_cam[1:]-y_cam[:-1])/(s[1:]-s[:-1])  # Calculate dy/ds
    grad[-1, :] = grad[-2, :]  # Deal with final point
    norm[:, 0] = -grad[:, 1]  # Calculate normal 1 as -1/grad2
    norm[:, 1] = grad[:, 0]

    return norm


def F_Make(Xi1, Xi2, controls_cam, controls_thk_x,
           controls_thk_y, Tte, Oval_frac):
    """Function that compiles a blade from reduced variables"""

    # Turning distribution
    Xi1 = np.radians(Xi1)
    Xi2 = np.radians(Xi2)
    # Generate camber line from bezier curve
    (x_cam, y_cam) = GEN_TURNING(Xi1, Xi2, controls_cam)

    s2 = np.zeros((len(x_cam)))  # Initialise camber length array
    for i in range(1, len(x_cam)):
        s2[i] = s2[i-1]+((x_cam[i]-x_cam[i-1])**2 +
                         (y_cam[i]-y_cam[i-1])**2)**0.5
    psi = s2/s2.max()

    psi_new = dist_vino(200, 0, 1, 1./2000, 1./500)

    x_cam = np.interp(psi_new, psi, x_cam)
    y_cam = np.interp(psi_new, psi, y_cam)
    psi = psi_new
    norm = calc_norm(x_cam, y_cam)

    thk = F_TF_bez(controls_thk_x, controls_thk_y, Tte, Oval_frac)

    Z_U = y_cam+thk*norm[:, 1]	 # Apply thickness onto camber line for upper
    Z_L = y_cam-thk*norm[:, 1]	 # Apply thickness onto camber line for lower
    X_U = x_cam+thk*norm[:, 0]	 # Repeat upper for x rather than y
    X_L = x_cam-thk*norm[:, 0]

    return(X_L, Z_L, X_U, Z_U)

The Profile function is the main function which is called to generate a profile. Again this is largely from CJC, however there are additions from WD for a rounded trailing edge. The first part includes controls for the blade camber, leading edge radius, trailing edge thickness, boat tailing engle, oval fraction and thickness. 

In [ ]:
def Profile(X1, X2, TKTE, Cx, points=500, TE_points=200):
    """Function based on profgen.f (JDD) - GENERATES SIMPLE BLADE PROFILES"""

    # THIS FUNCTION SHOULD BE REPLACED AT SOME POINT TO USE
    # A BETTER BLADE SECTION GENERATOR
    # EXCUSE THE WEIRD FORMULATION, THIS IS CONVERSION FROM FORTRAN
    # USER HARD CODED VALUES BELOW - COULD BE TAKEN OUTSIDE FUNCTION CJC
    # NOTE: XP/XS ETC DON'T ALWAYS REFER TO THE PRESSURE/SUCTION SURFACEC=S
    # 'S' VARIABLES ARE THE LOWER SURFACE AND 'P' THE UPPER, SO IF EXIT ANGLE
    # IS GREATER THAN INLET THIS WILL BE THE SUCTION AND PRESSURE SURFACES

    controls_cam = [-0.0, -0.5, -0.8]
    Rle = 0.05
    Tte = TKTE/Cx
    Beta_te = 4.0
    Oval_frac = 0.3
    controls_thk_x = [0.0, 0.5, 1.0]
    controls_thk_y = [(2*Rle)**0.5*(1-Oval_frac), 0.25,
                      np.tan(np.radians(Beta_te))+Tte/2]

The points are generated and scaled accordingly.

In [ ]:
    (XLIN, YLIN, XUIN, YUIN) = F_Make(X1, X2, controls_cam, controls_thk_x,
                                      controls_thk_y, Tte, Oval_frac)

    XIN = (XUIN+XLIN)/2
    YIN = (YUIN+YLIN)/2
    XScale = max(max(XUIN), max(XLIN))
    XUIN = XUIN/XScale
    XLIN = XLIN/XScale
    XIN = XIN/XScale

    yoffset = np.mean(YIN)
    # Center on peak thickness - DEFAULTS STACKING TO PEAK THICKNESS
    YUIN = (YUIN-yoffset)/XScale
    YLIN = (YLIN-yoffset)/XScale
    YIN = (YIN - yoffset)/XScale

    Xnew = np.zeros((len(XUIN)+len(XLIN)-1))
    Xnew[:200] = XUIN[::-1]
    Xnew[199:] = XLIN[:]

    Ynew = np.zeros((len(YUIN)+len(YLIN)-1))
    Ynew[:200] = YUIN[::-1]
    Ynew[199:] = YLIN[:]

    Snew = np.zeros((len(Ynew)))
    for i in range(1, len(Xnew)):
        Snew[i] = Snew[i-1]+((Xnew[i]-Xnew[i-1])**2 +
                             (Ynew[i]-Ynew[i-1])**2)**0.5

    for i in range(1, len(Xnew)):
        if Xnew[i] == Xnew.min():
            Sle = Snew[i]
            Xle = Xnew[i]
            break

    XUIN = np.interp(np.linspace(0, Sle, points), Snew, Xnew)[::-1]
    YUIN = np.interp(np.linspace(0, Sle, points), Snew, Ynew)[::-1]
    XLIN = np.interp(np.linspace(Sle, Snew[-1], points), Snew, Xnew)
    YLIN = np.interp(np.linspace(Sle, Snew[-1], points), Snew, Ynew)

    XScale = max(max(XUIN), max(XLIN))-Xle
    XUIN = (XUIN-Xle)/XScale
    XLIN = (XLIN-Xle)/XScale
    XIN = (XIN-Xle)/XScale

    # Center on peak thickness - DEFAULTS STACKING TO PEAK THICKNESS
    YUIN = (YUIN-yoffset)/XScale
    YLIN = (YLIN-yoffset)/XScale
    YIN = (YIN - yoffset)/XScale

To add a round trailing edge a circle is defined between the end of the suction and pressure surfaces. The points are then scaled again and output.

In [ ]:
    UTE_m = (XUIN[-2]-XUIN[-1])/(YUIN[-1]-YUIN[-2])
    LTE_m = (XLIN[-2]-XLIN[-1])/(YLIN[-1]-YLIN[-2])
    TE_circ_cx = (YUIN[-1]-YLIN[-1]+LTE_m*XLIN[-1] -
                  UTE_m*XUIN[-1])/(LTE_m-UTE_m)
    TE_circ_cy = YUIN[-1]+UTE_m*(TE_circ_cx-XUIN[-1])
    TE_circ_r = ((XLIN[-1]-TE_circ_cx)**2+(YLIN[-1]-TE_circ_cy)**2)**0.5
    U_theta = np.arccos((XUIN[-1]-TE_circ_cx)/TE_circ_r)
    L_theta = -np.arccos((XLIN[-1]-TE_circ_cx)/TE_circ_r)

    TE_points = 200
    TE_theta = np.linspace(0.95*U_theta, L_theta, TE_points)
    TEx = TE_circ_cx - TE_circ_r*np.cos(np.pi-TE_theta)
    TEy = TE_circ_cy + TE_circ_r*np.sin(np.pi-TE_theta)

    maxes = np.array([np.amax(XIN), np.amax(XUIN),
                      np.amax(XLIN), np.amax(TEx)])
    mins = np.array([np.amin(XIN), np.amin(XUIN), np.amin(XLIN)])
    cx = Cx*(np.amax(maxes)-np.amin(mins))

    XU = [cx*i for i in XUIN]
    YU = [cx*i for i in YUIN]
    XL = [cx*i for i in XLIN]
    YL = [cx*i for i in YLIN]
    TEx = [cx*i for i in TEx]
    TEy = [cx*i for i in TEy]

    te_index = int(points+TE_points/2)
    X = XU + TEx + XL[::-1]
    Y = YU + TEy + YL[::-1]
    XU = X[:te_index]
    YU = Y[:te_index]
    XL = X[te_index:]
    YL = Y[te_index:]

    return(XU, YU, XL, YL, X, Y)

### Length and area

This function finds the length of the suction surface and area of the profile. The number of points used can be adjusted for finer detail or faster calculations, but 40 is about right to get a good resolution with reasonable speed. The check at the start is to ensure the suction surface is measured, not the pressure surface, as the profile generation does distinguish between them. Standard library functions for length and area are used. 

In [ ]:
##############################
# SS LENGTH AND SECTION AREA #
##############################


def blade_dims(X1, X2, TE, Cx):
    """Return the suction surface length and section area of
    the normalised blade"""

    # Check if exit angle is less than the inlet angle. If so, invert them so
    # the Profile function returns XS and YS as the SS not the PS
    if X2 < X1:
        X2 = -X2
        X1 = -X1
    # Set number of points on the surfaces
    n_points = 40
    # Apply profile function to get the surface points
    surface = Profile(X1, X2, TE, Cx, points=n_points, TE_points=n_points)
    # Extract the two surfaces and offset them upwards
    # so all y values are positive
    min_y = np.amin(np.concatenate((surface[1], surface[3])))
    XP = surface[0]
    YP = [i+1.01*abs(min_y) for i in surface[1]]
    XS = surface[2]
    YS = [i+1.01*abs(min_y) for i in surface[3]]
    # Calculate the blade area by calculating the area under the pressure and
    # suction surfaces with Simpson's rule then finding the difference
    AP = simps(YP, x=XP)
    AS = simps(YS, x=XS)
    A_blade = AP-AS
    # Calculate the length of the suction surface
    ss_len = 0
    for i in range(n_points-1):
        ss_len += np.hypot(XS[i+1]-XS[i], YS[i+1]-YS[i])

    return A_blade, ss_len

### Make grid

This final function doesn't really need to be touched unless you change the parameters for the blade profile, in which case you can run it once to generate new csv files for interpolation. 

In [ ]:
def make_grid(t=0.0003, Cx=0.01):
    """Make .csv files with grids of normalised SS lengths and section areas"""

    # Define lists of variables
    xin = []
    xout = []
    sslen = []
    profile_area = []
    # Loop over reasonable ranges of inlet and exit angle, ensure exit is
    # greater than inlet for convenience
    for a1 in range(-60, 62, 2):
        for a2 in range(-80, 82, 2):
            x1 = a1
            x2 = a2
            if a2 < a1:
                x1 = -a1
                x2 = -a2
            # Calculate SS length and blade area, with a representative TE/Cx
            calc = blade_dims(x1, x2, t/Cx, 1.0)
            xin.append(a1)
            xout.append(a2)
            sslen.append(calc[1])
            profile_area.append(calc[0])
    # Add a blank space to the start of xin for the top left corner of the grid
    xout.insert(0, '')
    # Create a grid for the SS lengths
    with open('ss_grid.csv', mode='w') as ss_grid:
        table_writer = csv.writer(ss_grid, delimiter=',',
                                  quotechar='"', quoting=csv.QUOTE_MINIMAL)
        # The top row of exit angles
        table_writer.writerow(xout[:82])
        # Create other rows
        for i in range(int(len(xin)/81)):
            row = sslen[81*i:81*(i+1)]
            row.insert(0, xin[81*i])
            table_writer.writerow(row)
    # Create a grid for the areas
    with open('Profile_areas.csv', mode='w') as areas:
        table_writer = csv.writer(areas, delimiter=',',
                                  quotechar='"', quoting=csv.QUOTE_MINIMAL)
        # The top row of exit angles
        table_writer.writerow(xout[:82])
        # Create other rows
        for i in range(int(len(xin)/81)):
            row = profile_area[81*i:81*(i+1)]
            row.insert(0, xin[81*i])
            table_writer.writerow(row)